# Data Science Capstone Project

### Import Required libraries

In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

### Acquiring data from wikipedia with BeautifulSoup 

In [8]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(source,'html.parser')

### Inputing data into data frame

In [22]:
postal_code=[]
brough=[]
neigh=[]
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postal_code.append(cells[0].text)
        brough.append(cells[1].text)
        neigh.append(cells[2].text.rstrip(\n))

SyntaxError: unexpected character after line continuation character (<ipython-input-22-10d8a2e4c55c>, line 9)

In [21]:
col= ['Postal Code','Borough','Neighborhood']
df= pd.DataFrame(columns=col)
df['Postal Code']=postal_code
df['Borough']=brough
df['Neighborhood']=neigh
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
